### ERP image -> 6 tiles perspective image -> 6 tiles VMAF sacore

In [9]:
import subprocess
import json
import os
import pandas as pd
import numpy as np
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
#### SR 관련 라이브러리 ####
import torch
import torch.nn.functional as F
from torch.nn import Module, Conv2d, Sequential, Parameter
from torchvision.transforms import ToTensor, ToPILImage
import argparse
import sys
from multiprocessing import Process, Queue, set_start_method
import subprocess

from collections.abc import Iterator

from architect.model_factory.torch.owl_model_v1 import GeneratedSingleExitModelY

from custom_types.types import UpsampleType, BlockType, BlockInferenceType
from model.omnisrnet.modules.torch import SubPixelUpscaleModule, InformationMultiDistillationBlock, DenseBlock, ResidualBlock, \
    ProgressiveUpScaleModule, ChannelAttentionBlock
from model.omnisrnet.modules.torch import SubPixelUpscaleModuleY, ProgressiveUpScaleModuleY
from model.omnisrnet.modules.torch.function import initializer
#### OpenGL 관련 라이브러리 ####
os.environ['PYOPENGL_PLATFORM'] = 'egl'
from OpenGL import EGL
from OpenGL.GLES3 import *
import ctypes
from PIL import Image
import pyrr
import cv2 as cv
import time

from sphere import Sphere
from camera import Camera

env = os.environ.copy()
env['PATH'] = '/data/home/mckim/anaconda3/envs/Eagle/bin:' + env['PATH']
env['LD_LIBRARY_PATH'] = '/data/home/mckim/anaconda3/envs/Eagle/lib/x86_64-linux-gnu:' + env['LD_LIBRARY_PATH']


#### VMAF 측정 module

In [68]:

def calculate_vmaf(ref, dis):
    cmd = [
        'ffmpeg', '-hide_banner', '-loglevel', 'error',
        '-i', dis, # distorted
        '-i', ref, # reference
        '-filter_complex', f'[0:v]scale=1280x1280:flags=bicubic[distorted];[distorted][1:v]libvmaf=feature=name=psnr|name=float_ssim:model=path=/data/home/mckim/Eagle/VMAF-Tester/vmaf/model/vmaf_v0.6.1.json\\\\:enable_transform=true:log_fmt=json:log_path=/dev/stdout:n_threads=6',
        # '-filter_complex', '[0:v]scale=1280x720:flags=bicubic[distorted];[distorted][1:v]libvmaf=feature=name=psnr|name=float_ssim:model=path=/data/home/mckim/Eagle/VMAF-Tester/vmaf/model/vmaf_v0.6.1.json\\\\:enable_transform=true:log_fmt=json:log_path=/dev/stdout:n_threads=6',
        '-f', 'null', '-'
        ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, text=True, env=env)
    vmaf_result = json.loads(result.stdout)
    return {
        'reference':ref,
        'distorted':dis,
        'vmaf_mean':vmaf_result['pooled_metrics']['vmaf']['mean'],
        'vmaf_harmonic_mean':vmaf_result['pooled_metrics']['vmaf']['harmonic_mean'],
        }

In [57]:
dis = '/data/home/mckim/Eagle/360viwer_py/result_img/test_sphere_view_LR_(90,45).png'
dis = '/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/first_frame_1280x720_AREA_CV.png'
ref = '/data/home/mckim/Eagle/360viwer_py/result_img/test_sphere_view_(90,45).png'
ref = '/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/first_frame_1280x720_BICUBIC_PIL.png'
json_result = calculate_vmaf(ref, dis)
json_result

{'reference': '/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/first_frame_1280x720_BICUBIC_PIL.png',
 'distorted': '/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/first_frame_1280x720_AREA_CV.png',
 'vmaf_mean': 100.0,
 'vmaf_harmonic_mean': 100.0}

In [58]:
json_result['vmaf_mean']

100.0

#### 이미지 SR module

In [ ]:
def make_block_v5(block, input_channels) -> Sequential:
    block_type, block_info = block
    results = [None]
    if block_type == BlockType.IMD:
        results[-1] = InformationMultiDistillationBlock(
            layer_depth=block_info['layer_depth'],
            input_channels=input_channels,
            bottleneck_channels=block_info.get('bottleneck_channel'),
        )
    elif block_type == BlockType.Residual:
        results[-1] = ResidualBlock(
            layer_depth=block_info['layer_depth'],
            input_channels=input_channels,
            bottleneck_channels=block_info.get('bottleneck_channel'),
        )
    elif block_type == BlockType.Dense:
        results[-1] = DenseBlock(
            input_channels=input_channels,
            bottleneck_channels=block_info.get('bottleneck_channel'),
            layer_depth=block_info['layer_depth'],
        )
    ca = block_info['channel_attention']
    if ca > 0:
        results.append(ChannelAttentionBlock(reduction=ca, input_channels=input_channels))
    return Sequential(*results)

# Multi-Exit -> Single-Exit -> YUV에서 Y-only!
class GeneratedSingleExitModelY(Module):
    def __init__(self, arch):
        super(GeneratedSingleExitModelY, self).__init__()
        self.arch = arch
        self.scale_factor = arch.get('scale_factor') # or 4
        self.hidden_channels = arch['block_channels']
        block_channels = arch['block_channels']
        self.block_length = len(arch['block'][1].keys()) + 1

        # Feature Extractor 0 (Input layer)
        self.blocks0 = Conv2d(
            in_channels=1, # 3 -> 1 (RGB -> Y)
            out_channels=block_channels,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding='same'
        )
        initializer(self.blocks0)

        # Feature Extractor 1 ~
        for b_idx, b in arch['block'][1].items():
            b_idx = int(b_idx)
            self.__setattr__(f'blocks{b_idx + 1}', make_block_v5(b, block_channels))
            if b[1]['block_inference_type'] == BlockInferenceType.FrameConjunction:
                pre_c = Conv2d(
                    in_channels=block_channels * 2,
                    out_channels=block_channels,
                    kernel_size=(3, 3),
                    padding='same'
                )
                post_c = Conv2d(
                    in_channels=block_channels * 2,
                    out_channels=block_channels,
                    kernel_size=(3, 3),
                    padding='same'
                )
                initializer(pre_c)
                initializer(post_c)
                self.__setattr__(f'blocks{b_idx + 1}pre', pre_c)
                self.__setattr__(f'blocks{b_idx + 1}post', post_c)

        # Reconstructor for the final exit
        if arch['upsampling_type'] == UpsampleType.Progressive:
            self.upsample = ProgressiveUpScaleModuleY(
                upsampling_channels=arch['upsampling_channels'],
                scale_factor=self.scale_factor,
                input_channel=block_channels
            )
        else:
            self.upsample = SubPixelUpscaleModuleY(
                scale_factor=self.scale_factor,
                input_channel=block_channels
            )
        # self.__setattr__(f'upsample{b_idx + 1}', u)
        print("[DEBUG, owl_model_v1] Reconstructors", self.upsample)


    def frame_conjunction_params(self) -> Iterator[Parameter]:
        for p in self.rgb.parameters():
            yield p
        for p in self.frame_conjunction.parameters():
            yield p
        for p in self.frame_conjunction_pre.parameters():
            yield p
        for p in self.frame_conjunction_post.parameters():
            yield p

    def feature_extractor_params(self, layer_idx=-1) -> Iterator[Parameter]:
        if layer_idx == -1:
            # ALL
            for b in range(self.block_length):
                for p in self.__getattr__(f'blocks{b}').parameters():
                    yield p

        # Specific exit
        elif 0 < layer_idx < self.block_length:
            for p in self.__getattr__(f'blocks{layer_idx}').parameters():
                yield p

        else:
            return []

    def reconstructor_params(self) -> Iterator[Parameter]:
        for p in self.upsample.parameters():
            yield p

    def exit_idx_params(self, idx) -> Iterator[Parameter]:
        for p in self.__getattr__(f'upsample{idx}').parameters():
            yield p
        for i in range(idx + 1):
            for p in self.__getattr__(f'blocks{i}').parameters():
                yield p

    def forward(self, x_input):
        B, T, C, H, W = x_input.size()
        out = x_input.view(B*T, C, H, W)
        
        # srs = []

        # biups
        biups = F.interpolate(out, scale_factor=self.scale_factor, mode='bilinear', align_corners=False)    # B*T C sH sW

        # Block 0~ + Exit 0~
        for b_idx in range(self.block_length):
            if not hasattr(self, f'blocks{b_idx}pre'):
                out2 = self.__getattr__(f'blocks{b_idx}')(out)
            else:
                out2 = out.view(B, T, -1, H, W)
                forward_hiddens, backward_hiddens = [], []
                for i in range(T):
                    forward = out2[:, i]
                    backward = out2[:, T - i - 1]
                    outf = forward
                    outb = backward
                    if i > 0:
                        outf = torch.cat([outf, forward_hiddens[-1]], dim=1)
                        outb = torch.cat([outb, backward_hiddens[-1]], dim=1)
                        outf = self.__getattr__(f'blocks{b_idx}pre')(outf)
                        outb = self.__getattr__(f'blocks{b_idx}pre')(outb)
                    outf = self.__getattr__(f'blocks{b_idx}')(outf) # + forward    # B (bc) H W
                    outb = self.__getattr__(f'blocks{b_idx}')(outb) # + backward    # B (bc) H W
                    forward_hiddens.append(outf)
                    backward_hiddens.append(outb)
                res = []
                for i in range(T):
                    forward = torch.cat([forward_hiddens[i], backward_hiddens[T - i - 1]], dim=1)    # B 2(bc) H W
                    forward = self.__getattr__(f'blocks{b_idx}post')(forward)    # B (bc) H W
                    res.append(forward)
                out2 = torch.stack(res, dim=1)    # B T (bc) H W # Skip Connection
                out2 = out2.view(B * T, -1, H, W)    # B*T (bc) H W
            if b_idx > 0:
                out = out2 + out
            else:
                out = out2

#             print("[DEBUG, anytime_model_v5] (forward) feature extractors > b_idx, out.shape:", b_idx, out.shape)
#             print("[DEBUG, anytime_model_v5] B, T, C, H, W, self.scale_factor:", B, T, C, H, W, self.scale_factor)
#             # print("[DEBUG, anytime_model_v5], sH sW:", B, T, C, self.scale_factor * H, self.scale_factor * W)
            
#             print("[DEBUG, anytime_model_v5] self.__getattr__(f'upsample{b_idx}')(out).shape:", self.__getattr__(f'upsample{b_idx}')(out).shape)
#             print("[DEBUG, anytime_model_v5] biups.shape:", biups.shape)

        # Upsample
        # u = self.__getattr__(f'upsample{b_idx}')(out) + biups # B*T C sH sW
        u = self.upsample(out) + biups # B*T C sH sW
        u = torch.clamp(u, 0, 1) # tf.clip_by_value(u, 0, 1)
        u = u.view(B, T, C, self.scale_factor * H, self.scale_factor * W)    # B T C sH sW
        return u

In [20]:
frames = [[1,2,3],[3,4,5],[5,6,7],[7,8,9]]
# check frames is empty
while True:
    if not frames:
        print("empty")
        break
    else:
        print(frames.pop(0))

[1, 2, 3]
[3, 4, 5]
[5, 6, 7]
[7, 8, 9]
empty


In [76]:
def sr_init(exit_num):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("[DEBUG] device", device)
    
    file_path = f'./model/owlnet/archs_x4.json'
    with open(file_path, 'r') as f:
        archs = json.load(f)
        print(archs[int(exit_num)-1]) # [0] ~ [4]까지 있음 각각 exit1 ~ exit5
    
    # torch 모델 경로. 위에 archs[i] 중 하나를 골라서 넣어야 함
    model_path = f'/data/home/mckim/Eagle/VMAF-Tester/Eagle_models/x4_Sub_12_exit{exit_num}_Dense_fixed_72_0_4_MODPerspective_general-01_ymodel_yuvdata_epoch10_hrdown1_lrdown4_maxframe9000.pt'
    # dummy 모델 생성
    model = GeneratedSingleExitModelY(archs[int(exit_num)-1]).cuda()
    # torch 모델 로드
    state_dict = torch.load(model_path)
    # dummy 모델에 state_dict 로드
    model.load_state_dict(state_dict)
    # print("[DEBUG] model", model)
    
    # 모델을 gpu로 올림
    model = model.to(device)
    
    model.eval()
    
    return model, device

def inference(model, device, frames, sr_scale):
    height, width = frames[0].shape
    
    # frames = np.zeros((len(frames), int(height *3/2), width), dtype=np.uint8)
    # y_frames = np.zeros((len(frames), height , width), dtype=np.uint8)
    sr_frame = np.zeros((int(height  *sr_scale), width *sr_scale), dtype=np.uint8)

    y_frames = [frame[:height*2//3,:width] for frame in frames]
    y_frames = np.array(y_frames)
            
    input_images = torch.tensor(y_frames, dtype=torch.uint8)
    input_images = input_images.to(device)
    input_images = input_images.float() / 255.0
    # (B, 1, 1, H, W) 형태로 확장
    input_images = input_images.unsqueeze(1).unsqueeze(1)  # 채널, 배치 차원 추가
    # print("[DEBUG] input_image unsqueeze shape: ", input_images.shape)

    # print("[DEBUG] preprocess timee: ", (time.time() - start_inf)* 1000) # 5ms
    now = time.time()
    with torch.no_grad(): # 빼면 output_of_memory 에러 발생! inference할 때는 with torch.no_grad()로 감싸줘야 함!
        output_images = model(input_images)
    # print("[DEBUG] output_image shape: ", output_images.shape)
    print("[DEBUG] torch inference time: ", (time.time() - now)* 1000) # 25ms
    now = time.time()
    output_images = output_images.squeeze(1).squeeze(1)
    output_images = (output_images *255).to(device, dtype=torch.uint8)
    output_images = output_images.cpu().detach().numpy() # squeeze(0): 첫번째(index) 차원 제거, squeeze(): 크기가 1인 차원 제거
    print("[DEBUG] cpu detach time", (time.time() - now)* 1000) # 이게 왜 갑자기 오래걸리지?
    now = time.time()
    for i in range(len(frames)):
        # output_images[i] *= 255
        sr_frame = cv.resize(frames[i], (width *sr_scale, int(height *sr_scale)), interpolation=cv.INTER_NEAREST)
        sr_frame[:height*2//3 *sr_scale,:width *sr_scale] = output_images[i]
        
        sr_frame = cv.cvtColor(sr_frame, cv.COLOR_YUV2BGR_I420)
        cv.imwrite(f'Eagle_result/test_sphere_view_SR_({i}).png', sr_frame) ## 모든 delay는 저장이 원인. 이거 빼면 frame당 2ms 정도.
        print("[DEBUG] video write time per frame: ", (time.time() - now)* 1000) # 80~120ms



#### ERP -> 6 Tiles Opengl module

In [19]:
def load_texture(texture, width, height):
    glBindTexture(GL_TEXTURE_2D, texture)
    # Set the texture wrapping parameters
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    # Set texture filtering parameters
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, width, height, 0, GL_RGBA, GL_UNSIGNED_BYTE, None)
    return texture

def compile_shader(source, shader_type):
    shader = glCreateShader(shader_type)
    glShaderSource(shader, source)
    glCompileShader(shader)
    
    success = glGetShaderiv(shader, GL_COMPILE_STATUS)
    if not success:
        infoLog = glGetShaderInfoLog(shader)
        print(f"Shader compile error: {infoLog}")
    
    return shader

def create_program(vertex_src, fragment_src):
    vertex_shader = compile_shader(vertex_src, GL_VERTEX_SHADER)
    fragment_shader = compile_shader(fragment_src, GL_FRAGMENT_SHADER)
    program = glCreateProgram()
    glAttachShader(program, vertex_shader)
    glAttachShader(program, fragment_shader)
    glLinkProgram(program)
    glDeleteShader(vertex_shader)
    glDeleteShader(fragment_shader)
    return program

def create_opengl_context():
    # 컨텍스트 생성 및 바인딩
    config_attribs = [
        EGL.EGL_SURFACE_TYPE, EGL.EGL_PBUFFER_BIT, 
        EGL.EGL_BLUE_SIZE, 8, 
        EGL.EGL_GREEN_SIZE, 8, 
        EGL.EGL_RED_SIZE, 8, 
        EGL.EGL_ALPHA_SIZE, 8,
        EGL.EGL_RENDERABLE_TYPE, EGL.EGL_OPENGL_ES3_BIT, 
        EGL.EGL_NONE
    ]
    context_attribs = [
        EGL.EGL_CONTEXT_CLIENT_VERSION, 3, 
        EGL.EGL_NONE
    ]
    # EGL 컨텍스트 설정 및 초기화
    display = EGL.eglGetDisplay(EGL.EGL_DEFAULT_DISPLAY)
    EGL.eglInitialize(display, None, None)
    EGL.eglBindAPI(EGL.EGL_OPENGL_ES_API)
    num_configs = EGL.EGLint()
    config = EGL.EGLConfig()
    EGL.eglChooseConfig(display, config_attribs, config, 1, num_configs)
    egl_context = EGL.eglCreateContext(display, config, EGL.EGL_NO_CONTEXT, context_attribs)
    EGL.eglMakeCurrent(display, EGL.EGL_NO_SURFACE, EGL.EGL_NO_SURFACE, egl_context)
    return display, egl_context

def view_change(program, view):
    glUniformMatrix4fv(glGetUniformLocation(program, "view"), 1, GL_FALSE, view)

def init_viewport(program, width, height):
    glViewport(0, 0, width, height)
    
    projection = pyrr.matrix44.create_perspective_projection_matrix(90, width / height, 0.1, 100)
    glUniformMatrix4fv(glGetUniformLocation(program, "projection"), 1, GL_FALSE, projection)

    view = pyrr.matrix44.create_look_at(
        pyrr.Vector3([0.0, 0.0, 0.0]), 
        pyrr.Vector3([0.0, 0.0, -1.0]), 
        pyrr.Vector3([0.0, 1.0, 0.0])
    )
    glUniformMatrix4fv(glGetUniformLocation(program, "view"), 1, GL_FALSE, view)

    model = pyrr.matrix44.create_identity(dtype=np.float32)
    glUniformMatrix4fv(glGetUniformLocation(program, "model"), 1, GL_FALSE, model)


def destroy_opengl_context(display, egl_context):
    EGL.eglDestroyContext(display, egl_context)
    EGL.eglTerminate(display)

def opengl_init(sphere, view_width, view_height, erp_width, erp_height):
    
    vertex_src = """
    attribute vec4 a_position;
    attribute vec2 a_texCoord;

    varying vec2 v_texCoord;

    uniform mat4 projection;
    uniform mat4 view;
    uniform mat4 model;
    
    void main() {
        gl_Position = projection * view * model * a_position;
        v_texCoord = a_texCoord;
    }
    """

    fragment_src = """
    precision mediump float;
    varying vec2 v_texCoord;
    uniform sampler2D texture;
    void main() {
       gl_FragColor = texture2D(texture, v_texCoord);
    }
    """

    program = create_program(vertex_src, fragment_src)
    glUseProgram(program)

    VAO = glGenVertexArrays(1)
    glBindVertexArray(VAO)

    VBO = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, VBO)
    glBufferData(GL_ARRAY_BUFFER, sphere.vertices.nbytes, sphere.vertices, GL_STATIC_DRAW)

    posotion_loc = glGetAttribLocation(program, "a_position")
    glEnableVertexAttribArray(posotion_loc)
    glVertexAttribPointer(posotion_loc, 3, GL_FLOAT, GL_FALSE, 12, ctypes.c_void_p(0))

    texCoodBuffer = glGenBuffers(1)
    glBindBuffer(GL_ARRAY_BUFFER, texCoodBuffer)
    glBufferData(GL_ARRAY_BUFFER, sphere.texCoords.nbytes, sphere.texCoords, GL_STATIC_DRAW)

    texCoord_loc = glGetAttribLocation(program, "a_texCoord")
    glEnableVertexAttribArray(texCoord_loc)
    glVertexAttribPointer(texCoord_loc, 2, GL_FLOAT, GL_FALSE, 0, ctypes.c_void_p(0))
    
    EVO = glGenBuffers(1)
    glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EVO)
    glBufferData(GL_ELEMENT_ARRAY_BUFFER, sphere.indexCount*8, sphere.indices, GL_STATIC_DRAW)
    
    # texture load
    sphereTexture = glGenTextures(1)
    load_texture(sphereTexture, erp_width, erp_height) # HR texture
    
    # texture load
    sphereLRTexture = glGenTextures(1)
    load_texture(sphereLRTexture, erp_width//4, erp_height//4) # LR texture: HR의 1/4 크기

    framebuffer = glGenFramebuffers(1)
    glBindFramebuffer(GL_FRAMEBUFFER, framebuffer)

    renderbuffer = glGenRenderbuffers(1)
    glBindRenderbuffer(GL_RENDERBUFFER, renderbuffer)

    # 렌더버퍼 생성 및 프레임버퍼에 연결
    fbo_texture = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, fbo_texture)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, view_width, view_height, 0, GL_RGB, GL_UNSIGNED_BYTE, None)
    glFramebufferTexture2D(GL_FRAMEBUFFER, GL_COLOR_ATTACHMENT0, GL_TEXTURE_2D, fbo_texture, 0);  
    # 렌더버퍼 생성 및 프레임버퍼에 연결
    glRenderbufferStorage(GL_RENDERBUFFER, GL_DEPTH24_STENCIL8, view_width, view_height)
    glFramebufferRenderbuffer(GL_FRAMEBUFFER, GL_DEPTH_STENCIL_ATTACHMENT, GL_RENDERBUFFER, renderbuffer)
    
    lrframebuffer = glGenFramebuffers(1)
    glBindFramebuffer(GL_FRAMEBUFFER, lrframebuffer)

    lrrenderbuffer = glGenRenderbuffers(1)
    glBindRenderbuffer(GL_RENDERBUFFER, lrrenderbuffer)

    # 렌더버퍼 생성 및 프레임버퍼에 연결
    lrfbo_texture = glGenTextures(1)
    glBindTexture(GL_TEXTURE_2D, lrfbo_texture)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, view_width//4, view_height//4, 0, GL_RGB, GL_UNSIGNED_BYTE, None)
    glFramebufferTexture2D(GL_FRAMEBUFFER, GL_COLOR_ATTACHMENT0, GL_TEXTURE_2D, lrfbo_texture, 0);  
    # 렌더버퍼 생성 및 프레임버퍼에 연결
    glRenderbufferStorage(GL_RENDERBUFFER, GL_DEPTH24_STENCIL8, view_width//4, view_height//4)
    glFramebufferRenderbuffer(GL_FRAMEBUFFER, GL_DEPTH_STENCIL_ATTACHMENT, GL_RENDERBUFFER, lrrenderbuffer)

    if glCheckFramebufferStatus(GL_FRAMEBUFFER) != GL_FRAMEBUFFER_COMPLETE:
        print("ERROR::FRAMEBUFFER:: Framebuffer is not complete!")
        exit()
    else:
        print("Framebuffer is complete!")
        
    return program, sphereTexture, sphereLRTexture, VAO, framebuffer, lrframebuffer

In [18]:
erp_height, erp_width = 2560, 5120
new_frame = cv.imread('Eagle_result/first_frame_5120x2560.png')
# new_frame = cv.flip(new_frame, 0)
# new_frame = cv.cvtColor(new_frame, cv.COLOR_BGR2RGBA)
startTime = time.time()
new_frame_LR = cv.resize(new_frame ,(erp_width//4, erp_height//4), interpolation=cv.INTER_AREA)
cv.imwrite('Eagle_result/first_frame_1280x720_AREA_CV.png', new_frame_LR)
print("cv resize time: ", time.time() - startTime)

cv resize time:  0.058817148208618164


#### Main
1. 4K ERP 이미지가 INPUT
2. 4K ERP -> 6 Tiles perspective 이미지로 변환(HR)
3. 1/4 size ERP -> 6 Tiles perspective 이미지로 변환(LR)
4. 6 Tiles x4 SR(SR)
5. 6 Tiles VMAF score 측정 (HR <-> SR)

In [80]:
if __name__=='__main__':
    # parser = argparse.ArgumentParser(
    #     description="Video super resolution TEST using Pytorch",
    # )
    # parser.add_argument("-i", help="input path", default="Eagle_result/first_frame_5120x2560.png")
    # # parser.add_argument("-o", help="output path", default="torch_result.mp4")
    # parser.add_argument("-gpu", help="gpu device num", default="7")
    # parser.add_argument("-exit_num", help="model exit num 1~5", default="5")

    # args = parser.parse_args()
    
    # os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
    env["CUDA_VISIBLE_DEVICES"] = '4'
    ## HR 기준 width, height
    erp_height, erp_width = 2560, 5120 # LR의 경우 720, 1280
    view_height, view_width = 720, 1280 # LR의 경우 180, 320
    view_height, view_width = 1280, 1280 # 8개 정사각형 tile로 나눌경우
    sr_scale = 4
    # exit_num = args.exit_num
    
    model, device = sr_init(exit_num=1)
    lr_frames = []
    
    # create opengl context
    display, egl_context = create_opengl_context()
    # create sphere
    sphere = Sphere()
    # create camera
    camera = Camera()
    
    views = [(45,45), (135,45), (225,45), (315,45), (45,-45), (135,-45), (225,-45), (315,-45)]
    # views = [(0,0), (90,0), (180,0), (270,0), (0,90), (0,-90)]
    directions = []
    for jaw, pitch in views:
        camera.update_camera_vectors(pitch, jaw)
        directions.append((jaw, pitch, camera.get_view_matrix()))
        
    # init opengl
    program, sphereTexture, sphereLRTexture, VAO, framebuffer, lrframebuffer  = opengl_init(sphere, view_width, view_height, erp_width, erp_height)
        
    glUseProgram(program)
    glClearColor(0., 0.25, 0.25, 1)
    
    # load image
    # new_frame = cv.imread(args.i)
    new_frame = cv.imread('Eagle_result/first_frame_5120x2560.png')
    new_frame = cv.flip(new_frame, 0)
    new_frame = cv.cvtColor(new_frame, cv.COLOR_BGR2RGBA)
    startTime = time.time()
    new_frame_LR = cv.resize(new_frame ,(erp_width//4, erp_height//4), interpolation=cv.INTER_AREA)
    print("cv resize time: ", time.time() - startTime)

    glBindVertexArray(VAO)

    image_data = np.zeros((view_height, view_width, 3), dtype=np.uint8)
    LR_image_data = np.zeros((view_height//4, view_width//4, 3), dtype=np.uint8)
    
    startTime = time.time()

    glBindFramebuffer(GL_FRAMEBUFFER, framebuffer)
    glBindTexture(GL_TEXTURE_2D, sphereTexture)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, new_frame.shape[1], new_frame.shape[0], 0, GL_RGBA, GL_UNSIGNED_BYTE, new_frame.data)
    
    init_viewport(program, view_width, view_height)
    ### HR rendering and save ###
    for jaw, pitch, view in directions:
        view_change(program, view)
        #### HR image save ####
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
        
        glDrawElements(GL_TRIANGLES, sphere.indexCount, GL_UNSIGNED_SHORT, None)

        glReadPixels(0, 0, view_width, view_height, GL_RGB, GL_UNSIGNED_BYTE, image_data)

        image_data = cv.cvtColor(image_data, cv.COLOR_RGB2BGR)

        # 해상도가 커서 저장이 오래걸림.. 저장하는거 빼면 0.1초 미만
        cv.imwrite(f'Eagle_result/test_sphere_view_({jaw},{pitch}).png', image_data)
    print("HR tiles image saved, time: ", time.time() - startTime)
    
    startTime = time.time()
    
    glBindFramebuffer(GL_FRAMEBUFFER, lrframebuffer)
    glBindTexture(GL_TEXTURE_2D, sphereLRTexture)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, new_frame_LR.shape[1], new_frame_LR.shape[0], 0, GL_RGBA, GL_UNSIGNED_BYTE, new_frame_LR.data)
    
    init_viewport(program, view_width//4, view_height//4)
    ### LR rendering and save ###
    for jaw, pitch, view in directions:
        view_change(program, view)
        #### LR image save ####
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

        glDrawElements(GL_TRIANGLES, sphere.indexCount, GL_UNSIGNED_SHORT, None)

        glReadPixels(0, 0, view_width//4, view_height//4, GL_RGB, GL_UNSIGNED_BYTE, LR_image_data)

        lr_frames.append(cv.cvtColor(LR_image_data, cv.COLOR_RGB2YUV_I420))
        # LR_image_data = cv.cvtColor(LR_image_data, cv.COLOR_RGB2BGR)

        # cv.imwrite(f'Eagle_result/test_sphere_view_LR_({jaw},{pitch}).png', LR_image_data)
    print("LR tiles image saved, time: ", time.time() - startTime)

    #### inference process ####
    startTime = time.time()
    inference(model, device, lr_frames, sr_scale)
    print("inference time: ", time.time() - startTime)
    
    #### VMAF calculation ####
    startTime = time.time()
    i = 0
    vmaf_scores = []
    vmaf_scores_lr = []
    for jaw, pitch, _ in directions:
        ref = f'/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/test_sphere_view_({jaw},{pitch}).png'
        dis = f'/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/test_sphere_view_SR_({i}).png'
        i += 1
        json_result = calculate_vmaf(ref, dis)
        vmaf_scores.append(json_result['vmaf_mean'])
        print('vmaf calculate time: ', time.time() - startTime)
        
        dis2 = f'/data/home/mckim/Eagle/VMAF-Tester/Eagle_result/test_sphere_view_LR_({jaw},{pitch}).png'
        json_result = calculate_vmaf(ref, dis2)
        vmaf_scores_lr.append(json_result['vmaf_mean'])

    print("vmaf_scores: ", vmaf_scores)
    print("vmaf_scores_LR: ", vmaf_scores_lr)
    print("vmaf_diff: ", np.array(vmaf_scores) - np.array(vmaf_scores_lr))
    print("vmaf overall time: ", time.time() - startTime)

    glBindFramebuffer(GL_FRAMEBUFFER, 0)
    glBindVertexArray(0)
    
    # destroy opengl context
    destroy_opengl_context(display, egl_context)

[DEBUG] device cuda
{'block': [4, {'0': ['Dense', {'block_inference_type': 'PerFrame', 'bottleneck_channel': 72, 'channel_attention': 0, 'layer_depth': 4, 'upsampling_channels': 8, 'upsampling_type': 'Sub'}], '1': ['Dense', {'block_inference_type': 'PerFrame', 'bottleneck_channel': 72, 'channel_attention': 0, 'layer_depth': 4, 'upsampling_channels': 8, 'upsampling_type': 'Sub'}], '2': ['Dense', {'block_inference_type': 'PerFrame', 'bottleneck_channel': 72, 'channel_attention': 0, 'layer_depth': 4, 'upsampling_channels': 8, 'upsampling_type': 'Sub'}], '3': ['Dense', {'block_inference_type': 'PerFrame', 'bottleneck_channel': 72, 'channel_attention': 0, 'layer_depth': 4, 'upsampling_channels': 8, 'upsampling_type': 'Sub'}]}], 'block_channels': 12, 'arch_name': 'x4_Sub_12_exit4_Dense_fixed_72_0_4', 'scale_factor': 4, 'upsampling_channels': 4, 'upsampling_type': 'Sub'}
[DEBUG, owl_model_v1] Reconstructors SubPixelUpscaleModuleY(
  (deconv): Conv2d(12, 16, kernel_size=(3, 3), stride=(1, 1), 